In [3]:
import pandas as pd
import requests
import csv
import time
import datetime
import config as config

In [4]:
# increase cell width
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [5]:
url = "https://developer.cumtd.com/api/v2.2/json/getdeparturesbystop?key=" + config.mtd_api_key + "&stop_id=iu:1&count=5"
payload= {}
headers = {}
response = requests.request("GET", url, headers=headers, data=payload)

print(response.json()["time"])
print(response.json()["departures"][0]["stop_id"])
print(response.json()["departures"][0]["headsign"])
print(response.json()["departures"][1]["trip"]["direction"])
print(response.json()["departures"][0]["location"])
print("Sceduled: " + str(response.json()["departures"][1]["scheduled"]))
print("Expected: " + str(response.json()["departures"][1]["expected"]))
print(str(response.json()["departures"][1]["expected_mins"]) + " mins")

2021-11-23T14:17:16-06:00
IU:1
12E Teal Orchard Downs
North
{'lat': 40.11535, 'lon': -88.241483}
Sceduled: 2021-11-23T14:28:00-06:00
Expected: 2021-11-23T14:32:55-06:00
16 mins


In [6]:
# 2021-10-23T18:05:36-05:00 -> 18:05:36 for 'soft', datetime for 'hard'
def time_converter(time, soft_hard):
    split_1 = time.split("T")
    split_2 = split_1[1].split("-")
    if soft_hard == "hard":
        split_3 = split_2[0].split(':')
        return datetime.datetime.now().replace(hour = int(split_3[0]), minute = int(split_3[1]), second = int(split_3[2]), microsecond = 0)
    elif soft_hard == "soft":
        return str(split_2[0])

# Returns a list of all buses, with different vehicle_id's, currently heading to this stop_id.
# When a bus leaves, they stop appearing in requests, and therefore the list everytime it's called.
def arrival_queue(stop_id):
    list = []
    response = requests.get("https://developer.cumtd.com/api/v2.2/json/getdeparturesbystop?key=" + config.mtd_api_key_2 + "&stop_id=" + stop_id)
    try:
        response.json()["departures"] # if "departures" does not exist, this stop doesn't exist either
    except KeyError:
        return False
    for i in range(0, len(response.json()["departures"])):
        if str(response.json()["departures"][i]["headsign"]) not in list: # Eliminating any dupes, leaving them for next time they're called
            list.append(str(response.json()["departures"][i]["headsign"]))
    print("Bus arrival queue: ")
    print(list)
    return list

#  key:'headsign', 0:'route_id', 1:'origin', 2:'destination', 3:'scheduled', 4:'expected', 5:'expected_mins', dictionary with info about current buses with same stop_id
def stop_id_info(stop_id):
    bus_dict = {}
    response = requests.get("https://developer.cumtd.com/api/v2.2/json/getdeparturesbystop?key=" + config.mtd_api_key_2 + "&stop_id=" + stop_id)
    try:
        response.json()["departures"]
    except KeyError:
        return False
    for i in range(0, len(response.json()["departures"])):
        if str(response.json()["departures"][i]["headsign"]) not in bus_dict.keys(): # Eliminating any dupes, leaving them for next time they're called
            bus_dict[str(response.json()["departures"][i]["headsign"])] = [str(response.json()["departures"][i]["route"]["route_id"]), str(response.json()["departures"][i]["origin"]["stop_id"]), str(response.json()["departures"][i]["destination"]["stop_id"]), str(response.json()["departures"][i]["scheduled"]), str(response.json()["departures"][i]["expected"]), str(response.json()["departures"][i]["expected_mins"])]
    return bus_dict

# Returns time difference when given stop_id, the bus_headsign, and the index that determines whether it is scheduled or expected
def time_difference(stop_id, bus_headsign, index, original_expected):
    response = requests.get("https://developer.cumtd.com/api/v2.2/json/getdeparturesbystop?key=" + config.mtd_api_key_2 + "&stop_id=" + stop_id)
    current_time = time_converter(response.json()["time"], 'hard')
    current_sc_ex = time_converter(original_expected, 'hard')
    return (current_time - current_sc_ex).total_seconds()

def start_csv():
    with open("bus_data.csv", "wt") as f:
        filewriter = csv.writer(f, delimiter=",")
        filewriter.writerow(["stop_id", "bus_headsign", "route_id", "expected_time", "true_time"])

def append_csv(stop_id, bus_headsign, route_id, expected_time, true_time):
    with open("bus_data.csv", "a") as f:
        filewriter = csv.writer(f, delimiter=",")
        filewriter.writerow([stop_id, bus_headsign, route_id, expected_time, true_time])
                
# Outcome of whether current several buses came before or after the scheduled or expected time -> 3 for scheduled, 4 or expected
def bulk_difference(stop_id, sc_ex):
    results = []
    bus_headsign = arrival_queue(stop_id)
    if bus_headsign is False:
        print(stop_id + " does not exist.")
        return
    start_csv()
    for i in range(0, 20): # change depending on how many departures want to be monitored, will run consecutively as a queue for each departure
        bus_headsign = arrival_queue(stop_id)
        original_expected = stop_id_info(stop_id) # this checks to see if there are departures
        if original_expected is False:
            print("No departures anytime soon for " + stop_id + ".")
            return
        try:
            original_expected = stop_id_info(stop_id)[bus_headsign[0]][4]
            current_routeid = stop_id_info(stop_id)[bus_headsign[0]][0]
        except KeyError:
            break
        while (True):
            if arrival_queue(stop_id)[0] != bus_headsign[0]:
                if len(results) != 0: # accounts for buses that arrive before the range, improbable but did happen
                    append_csv(stop_id, bus_headsign[0], current_routeid, time_converter(original_expected, "soft"), str(results[len(results) - 1]))
                    print('Arrived at: ' + str(results[len(results) - 1]))
                    break
                else: # eliminate this whole block later, was made for this particular print
                    print('Arrived at: ' + int(time_difference(stop_id, bus_headsign[0], sc_ex, original_expected)))
                    append_csv(stop_id, bus_headsign[0], current_routeid, time_converter(original_expected, "soft"), str(results[len(results) - 1]))
                    break
            if -120 <= int(time_difference(stop_id, bus_headsign[0], sc_ex, original_expected)) and int(time_difference(stop_id, bus_headsign[0], sc_ex, original_expected)) <= 300:
                print("Current time difference: " + str(time_difference(stop_id, bus_headsign[0], sc_ex, original_expected)))
                print("Expected time: " + original_expected + "\n")
                results.append(int(time_difference(stop_id, bus_headsign[0], sc_ex, original_expected)))
                time.sleep(3)
            else:
                print("Time until next bus arrives: " + str(time_difference(stop_id, bus_headsign[0], sc_ex, original_expected)))
                print("Expected time: " + original_expected + "\n")
                results.append(int(time_difference(stop_id, bus_headsign[0], sc_ex, original_expected)))
                time.sleep(30)
    print(results)
    
def main():
    bulk_difference("IU:1", 4)

if __name__ == "__main__":
    main()

#  Monitor first three in the queue

Bus arrival queue: 
['12E Teal Orchard Downs', '22N Illini Limited', '13N Silver', '5E Green Hopper', '2U Red', '21 Raven Vet-Med', '5E Green']
Bus arrival queue: 
['12E Teal Orchard Downs', '22N Illini Limited', '13N Silver', '5E Green Hopper', '2U Red', '21 Raven Vet-Med', '5E Green']
Bus arrival queue: 
['12E Teal Orchard Downs', '22N Illini Limited', '13N Silver', '5E Green Hopper', '2U Red', '21 Raven Vet-Med', '5E Green']
Time until next bus arrives: -349.0
Expected time: 2021-11-23T14:23:12-06:00

Bus arrival queue: 
['12E Teal Orchard Downs', '5E Green Hopper', '22N Illini Limited', '13N Silver', '2U Red', '21 Raven Vet-Med', '5E Green']
Time until next bus arrives: -318.0
Expected time: 2021-11-23T14:23:12-06:00

Bus arrival queue: 
['12E Teal Orchard Downs', '5E Green Hopper', '22N Illini Limited', '13N Silver', '2U Red', '21 Raven Vet-Med', '5E Green']
Time until next bus arrives: -287.0
Expected time: 2021-11-23T14:23:12-06:00

Bus arrival queue: 
['12E Teal Orchard Downs',

ConnectionError: HTTPSConnectionPool(host='developer.cumtd.com', port=443): Max retries exceeded with url: /api/v2.2/json/getdeparturesbystop?key=a1a6795b640b48988744a1a475588730&stop_id=IU:1 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001FB8F328CA0>: Failed to establish a new connection: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond'))

In [29]:
import folium
map = folium.Map(location=[40.10051, -88.222833], zoom_start = 11, tiles = 'cartodbpositron')
map
# https://vega.github.io/vega/examples/time-units/
# https://altair-viz.github.io/index.html

In [30]:
# CSV Format: stop_id | bus_headsign | route_id | expected_time | true_time

# At the start of the code:
with open("bus_data.csv", "wt") as f:
    filewriter = csv.writer(f, delimiter=",")
    filewriter.writerow(["stop_id", "bus_headsign", "route_id", "expected_time", "true_time"])

# To append to this same CSV:
with open("bus_data.csv", "a") as f:
    filewriter = csv.writer(f, delimiter=",")
    filewriter.writerow(["IU:1", "220N Illini", "100 YELLOW", "15:36:19", "15:37:23"])


In [31]:
# Changes to be done:
# Code is not running properly when the stop doesn't have any expected departures.
# Check for any index errors that might occur, like in the result break.
# We need to maintain the original expected time so we can get the real result.

response = requests.get("https://developer.cumtd.com/api/v2.2/json/getdeparturesbystop?key=" + config.mtd_api_key_2 + "&stop_id=" + 'WDSR:2')
if response.json()["departures"] == []:
    print('This stop has no departures.')
else:
    print('Passing here')
    response.json()

Passing here


In [54]:
response = requests.get("https://developer.cumtd.com/api/v2.2/json/getdeparturesbystop?key=" + config.mtd_api_key_2 + "&stop_id=" + 'PLAZA')
response.json()

{'time': '2021-11-22T14:15:30-06:00',
 'new_changeset': True,
 'status': {'code': 200, 'msg': 'ok'},
 'rqst': {'method': 'GetDeparturesByStop', 'params': {'stop_id': 'PLAZA'}},
 'departures': [{'stop_id': 'PLAZA:3',
   'headsign': '22S Illini',
   'route': {'route_color': '5a1d5a',
    'route_id': 'ILLINI LIMITED',
    'route_long_name': 'Illini Limited',
    'route_short_name': '22',
    'route_text_color': 'ffffff'},
   'trip': {'trip_id': '[@6.0.14327389@][2][1622152086585]/70__I2_NONUIMF',
    'trip_headsign': 'Transit Plaza',
    'route_id': 'ILLINI LIMITED',
    'block_id': 'I2 NONUIMF',
    'direction': 'South',
    'service_id': 'I2 NONUIMF',
    'shape_id': '[@6.0.14327389@]15'},
   'vehicle_id': '2017',
   'origin': {'stop_id': 'LNCLNKLRNY:3'},
   'destination': {'stop_id': 'ARC:2'},
   'is_monitored': True,
   'is_scheduled': True,
   'is_istop': True,
   'scheduled': '2021-11-22T14:13:00-06:00',
   'expected': '2021-11-22T14:16:53-06:00',
   'expected_mins': 1,
   'location